In [2]:
import webbrowser
import json
import requests
import pandas as pd
from stravalib.client import Client
import pprint as pprint
import time

In [102]:
def initial_authorisation_token():

  # client id and client secret can be given as string literals, but it 
  # is recommended to set them as environment variables instead, we can 
  # then access them as shown here
  client_id = '139812'
  client_secret = '02fd7a653d36e750fcabbcb3e1f4717c65aa8e38'

  # url the user is directed to once they have logged in
  redirect_uri = 'http://localhost:8000'
   
  # send request to server to authorize a user
  # this will prompt the user to sign into strava and grant our application permissions
  request_url = f'http://www.strava.com/oauth/authorize?client_id={client_id}' \
                        f'&response_type=code&redirect_uri={redirect_uri}' \
                        f'&approval_prompt=force' \
                        f'&scope=profile:read_all,activity:read_all'
  
  # open url in browser
  webbrowser.open(request_url)
  
  # recieve code once user has logged in
  code = input('Insert the code from the url: ')
      
  # Get the access token
  token = requests.post(url='https://www.strava.com/api/v3/oauth/token',
                       data={'client_id': client_id,
                             'client_secret': client_secret,
                             'code': code,
                             'grant_type': 'authorization_code'})
  token = token.json()
 
  # save token for later (function shown below)
  write_token(token)

  return token

In [78]:
client_id, client_secret = open('client_secrets.txt').read().strip().split(',')
client = Client()
request_scope = ["read_all", "profile:read_all", "activity:read_all"]

redirect_url = "http://127.0.0.1:5000/authorization"

url = client.authorization_url(
    client_id=client_id,
    redirect_uri=redirect_url,
    scope=request_scope,
)


  client_id, client_secret = open('client_secrets.txt').read().strip().split(',')



In [76]:
# # Open the url that you created above in a web browser
# webbrowser.open(url)
# print(
#     """You will see a URL that looks like this:
#     http://127.0.0.1:5000/authorization?state=&code=12323423423423423423423550&scope=read,activity:read_all,profile:read_all,read_all
#     Copy the values between code= and & in the URL that you see in the browser."""
# )


In [46]:
# # Using input allows you to copy the code into your Python console (or Jupyter Notebook)
# code = input("Please enter the code that you received: ")

# # Exchange the code returned from Strava for an access token
# token_response = client.exchange_code_for_token(
#     client_id=client_id, client_secret=client_secret, code=code
# )


In [47]:
# # Save the token response as a JSON file
# json_path = 'strava_token_local.json'
# with open(json_path, "w") as f:
#     json.dump(token_response, f)

# print("Token saved - hooray!")

# # Access and refresh tokens
# access_token = token_response["access_token"]
# refresh_token = token_response["refresh_token"]


In [103]:
def read_token():

  try:
      with open('strava_token_local.json', 'r') as f:
          token = json.load(f)
          print(token)
  except FileNotFoundError:

      # token cannot be found, so cannot be refreshed
      # instead, follow the original authorisation procedure again
      token = initial_authorisation_token()

  return token

In [104]:
def write_token(token):

  with open('strava_token_local.json', 'w') as file:
    json.dump(token, file)

In [105]:
if token['expires_at'] < time.time():
    print("token expired")
else:
    print("Valid Token")

Valid Token


In [106]:
def refresh_token(token):

    # check if the token has expired
    if token['expires_at'] < time.time():
        print("Token not expired")
        try:
            # request a new access token using the refresh token
            token = requests.post(url='https://www.strava.com/api/v3/oauth/token',
                                data={'client_id': client_id,
                                    'client_secret': client_secret,
                                    'grant_type': 'refresh_token',
                                    'refresh_token': token['refresh_token']})
            token = token.json()
            write_token(token)
        except Exception as e:
            print("Error refreshing token:", e)
        
    return token

In [107]:
token = read_token()
token = refresh_token(token)
print(token)

{'access_token': '1f50c60ea5c15082d4aa0f589e890017400fa065', 'refresh_token': 'b5a3bc523c03c3f1f0fdb5cac68f1a6b83f91e53', 'expires_at': 1758777388}
{'access_token': '1f50c60ea5c15082d4aa0f589e890017400fa065', 'refresh_token': 'b5a3bc523c03c3f1f0fdb5cac68f1a6b83f91e53', 'expires_at': 1758777388}


In [110]:
token

{'access_token': '1f50c60ea5c15082d4aa0f589e890017400fa065',
 'refresh_token': 'b5a3bc523c03c3f1f0fdb5cac68f1a6b83f91e53',
 'expires_at': 1758777388}

In [114]:
token = read_token()
token = refresh_token(token)


# import all user activities

activities = []
page = 1
response = []

while True:
    
    # request new page of activities
    endpoint = f"https://www.strava.com/api/v3/athlete/activities?" \
                  f"access_token={token['access_token']}&" \
                  f"page={page}&" \
                  f"per_page=50"
    
    response = requests.get(endpoint).json()

    # check if page contains activities
    if len(response):

        # retrieve some fields for each activity
        # you can see the full list of fields by looking at the response json
        print(response)

    else:
      break  

# convert our activities to a DataFrame
df = pd.DataFrame(activities)

{'access_token': '1f50c60ea5c15082d4aa0f589e890017400fa065', 'refresh_token': 'b5a3bc523c03c3f1f0fdb5cac68f1a6b83f91e53', 'expires_at': 1758777388}
{'message': 'Rate Limit Exceeded', 'errors': [{'resource': 'Application', 'field': 'read rate limit', 'code': 'exceeded'}]}
{'message': 'Rate Limit Exceeded', 'errors': [{'resource': 'Application', 'field': 'read rate limit', 'code': 'exceeded'}]}
{'message': 'Rate Limit Exceeded', 'errors': [{'resource': 'Application', 'field': 'read rate limit', 'code': 'exceeded'}]}
{'message': 'Rate Limit Exceeded', 'errors': [{'resource': 'Application', 'field': 'read rate limit', 'code': 'exceeded'}]}
{'message': 'Rate Limit Exceeded', 'errors': [{'resource': 'Application', 'field': 'read rate limit', 'code': 'exceeded'}]}
{'message': 'Rate Limit Exceeded', 'errors': [{'resource': 'Application', 'field': 'read rate limit', 'code': 'exceeded'}]}
{'message': 'Rate Limit Exceeded', 'errors': [{'resource': 'Application', 'field': 'read rate limit', 'code'

KeyboardInterrupt: 

In [3]:
norm = pd.read_csv('normalised.csv')

In [5]:
pd.set_option('display.max_columns', None)
norm

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,id,start_date,start_date_local,timezone,utc_offset,location_city,location_state,location_country,achievement_count,kudos_count,comment_count,athlete_count,photo_count,trainer,commute,manual,private,visibility,flagged,gear_id,start_latlng,end_latlng,average_speed,max_speed,average_cadence,has_heartrate,average_heartrate,max_heartrate,heartrate_opt_out,display_hide_heartrate_option,elev_high,elev_low,upload_id,upload_id_str,external_id,from_accepted_tag,pr_count,total_photo_count,has_kudoed,athlete.id,athlete.resource_state,map.id,map.summary_polyline,map.resource_state
0,2,Evening Run,5008.3,1989,2018,29.3,Run,Run,NaN,15945142274,2025-09-26T17:44:48Z,2025-09-26T18:44:48Z,(GMT+00:00) Europe/London,3600.0,NaN,NaN,NaN,1,1,0,1,0,False,False,False,False,everyone,False,NaN,"[51.54412, -0.185391]","[51.544242, -0.188505]",2.518,4.257,76.2,True,169.0,197.0,False,True,47.6,35.9,17025869650,17025869650,garmin_ping_484501393060,False,1,0,False,25319543,1,a15945142274,_fryHxtc@AtBDl@Ll@H@TIZ?hANf@BlAPLGT?BDFBh@L^C...,2
1,2,Afternoon Run,4010.0,1728,3645,42.8,Run,Run,NaN,15924054783,2025-09-24T16:51:23Z,2025-09-24T17:51:23Z,(GMT+00:00) Europe/London,3600.0,NaN,NaN,NaN,0,1,0,1,0,False,False,False,False,everyone,False,NaN,"[51.544027, -0.185299]","[51.544336, -0.184491]",2.321,3.638,76.5,True,150.0,179.0,False,True,77.4,41.5,17004311183,17004311183,garmin_ping_483800288065,False,0,0,False,25319543,1,a15924054783,ooryHprb@Kq@M]QU?Oe@s@Wm@Oa@Kc@Yc@Ey@_@u@Mk@Lk...,2
2,2,Evening Run,2006.7,694,699,8.2,Run,Run,NaN,15903095207,2025-09-22T17:57:42Z,2025-09-22T18:57:42Z,(GMT+00:00) Europe/London,3600.0,NaN,NaN,NaN,1,1,0,1,0,False,False,False,False,everyone,False,NaN,"[51.546639, -0.184894]","[51.54411, -0.182697]",2.891,4.267,76.5,True,181.6,192.0,False,True,51.1,37.6,16982834169,16982834169,garmin_ping_483151230415,False,1,0,False,25319543,1,a15903095207,_vryHtlb@Nu@\aAXi@BDDCRg@x@{ARQXHd@HTIb@HPAf@B...,2
3,2,Evening Run,3009.0,1101,1115,19.1,Run,Run,NaN,15857721289,2025-09-18T18:57:52Z,2025-09-18T19:57:52Z,(GMT+00:00) Europe/London,3600.0,NaN,NaN,NaN,0,1,0,1,0,False,False,False,False,everyone,False,NaN,"[51.544128, -0.186059]","[51.544107, -0.182601]",2.733,4.117,76.7,True,170.9,200.0,False,True,55.6,37.6,16936199081,16936199081,garmin_ping_481711937291,False,0,0,False,25319543,1,a15857721289,qfryHrxc@EzA@Ba@BIJIB[@WD_@GGMQCQNGLCTEzACdC@\...,2
4,2,Evening Run,3007.7,1194,1202,19.1,Run,Run,NaN,15822819329,2025-09-15T18:42:55Z,2025-09-15T19:42:55Z,(GMT+00:00) Europe/London,3600.0,NaN,NaN,NaN,0,1,0,1,0,False,False,False,False,everyone,False,NaN,"[51.544077, -0.185931]","[51.543851, -0.182464]",2.519,4.050,76.0,True,160.1,195.0,False,True,55.5,37.6,16900397145,16900397145,garmin_ping_480639095365,False,0,0,False,25319543,1,a15822819329,_fryHrwc@?lBMb@y@QMKU?c@J[OYRENCPD|CIfBBPAzB@~...,2
